### Masking input to DAOStarFind and (possibly) filtering results for input to DBSCANGroup

In [1]:
# dependancies 

%matplotlib notebook

import sys
import time
import os
import warnings

import numpy as np
import numpy.ma as ma

import matplotlib as mpl
import matplotlib.pyplot as plt

import fitsio
from astropy.io import fits

from photutils.aperture import CircularAperture
from photutils.utils import make_random_cmap

sys.path.insert(0, '/home/bekah/gphoton_working')
sys.path.insert(0, '/home/bekah/gphoton_working/gPhoton')

In [37]:
# for searching for eclipses near object, don't need to run unless adding to extended source list 
from gPhoton.search.core import eclipses_near_object

galaxies = eclipses_near_object("NGC6872",5000)
#galaxies
galaxies

NGC6872 position: RA 304.2350104994999, DEC -70.76807516544
28 eclipses found within 5000 asec of NGC6872.


,eclipse,obstype,legs,ra_max,ra_min,dec_max,dec_min,fuv_temp
0,11734,NGS,0,304.466730,304.415310,-70.775859,-70.793806,NaN
1,11742,NGS,0,304.467033,304.390502,-70.775929,-70.802204,NaN
2,11759,DIS,0,304.669369,304.558220,-70.801689,-70.828869,NaN
3,11828,NGS,0,304.438985,304.356082,-70.794539,-70.820701,NaN
4,11830,NGS,0,304.349865,304.261890,-70.790079,-70.815875,NaN
5,11831,NGS,0,304.482771,304.371603,-70.789932,-70.816264,NaN
6,12501,NGS,0,304.490443,304.083534,-70.792888,-70.824512,NaN
7,12502,NGS,0,304.504302,304.422999,-70.803041,-70.829387,NaN
8,16907,NGS,0,304.475642,304.399747,-70.770510,-70.795864,28.020515
9,16911,NGS,0,304.457938,304.380716,-70.782344,-70.807956,28.762224


##### extended_sources_eclipses = [("NGC0055","02025","NGS"),("NGC0055","13770","GII"),("NGC5128","00581","NGS"),("NGC0205","23403","DIS"),("NGC0205","02383","NGS"), \
                             ("NGC1365","08336"),("NGC1316","03239"),("NGC5322","26863"),("NGC3945","37289"),("NGC4429","04637"), \
                             ("IC152","02360"),("NGC1398","13551")]

comparing_sources = [("NGC0205",8261),("NGC0205","23403","DIS"),("NGC0205","02383","NGS")]

In [ ]:
# run gphoton2 on list of extended sources, get output source list 
from gPhoton.pipeline import execute_pipeline
    
    # TODO: should write something in here so that it doesn't run if the source list already exists, etc. 
for e in comparing_sources: 
        eclipse = e[1]
        execute_pipeline(
                eclipse,
                "NUV",
                depth=30,
                threads=4,
                local_root="test_data",
                recreate=False,
                aperture_sizes=[12.8, 51.2],
                write={"movie": False, "image": True},
                coregister_lightcurves=True,
                compression="rice",
                lil=True
            )

In [ ]:
# modified function for calling DAOStarFind from gphoton2 

from photutils.detection import DAOStarFinder
import pandas as pd 

# in the original call in gphoton2, the image is fed as: ["cnt"] / exptime, which divides by the sum of exp time 
# across all frames. 


def DAOFindCall(cnt_image): 
    """returns source table after calling daofind on the cnt image"""
    
    daofind = DAOStarFinder(fwhm=5, threshold=0.01, ratio=1, theta=0)  # changed threshold from 0.01, fwhm from 5
    source_table = daofind(cnt_image).to_pandas()
    
    return source_table 

In [ ]:
# function for reading in source images and converting to HDUL / clipping (called in loop for figure)

from gPhoton.reference import eclipse_to_paths
from gPhoton.coadd import zero_flag_and_edge

def image_reader(eclipseNum):
    # info 
    eclipse, band, depth, compression = eclipseNum, "NUV", 30, "rice"

    galex_file_paths = eclipse_to_paths(eclipse, '/home/bekah/glcat/exploration_notebooks/test_data', depth, compression)

    # image to hdul 
    hdul = fitsio.FITS(galex_file_paths[band]['image'])
    cnt, flag, edge = [hdu.read() for hdu in hdul[1:4]]

    # masking cnt image
    masked_cnt = zero_flag_and_edge(cnt, flag, edge)
    
    return masked_cnt


# clipping 
def centile_clip(image, centiles=(1, 99)):
    """
    simple clipping function that clips values above and below a given
    percentile range
    """
    finite = np.ma.masked_invalid(image)
    bounds = np.percentile(finite[~finite.mask].data, centiles)
    result = np.ma.clip(finite, *bounds)
    
    if isinstance(image, np.ma.MaskedArray):
        
        return result
    
    return result.data

In [ ]:
# function for running DBSCANGroup 

from astropy.table import QTable, Table, Column 
from photutils.psf.groupstars import DBSCANGroup

def grouping_sources(positions, eps): 
    
    starlist = Table()

    #x_0 = list(zip(*positions))[0]
   # y_0 = list(zip(*positions))[1]

    starlist['x_0'] = positions["xcentroid"]
    starlist['y_0'] = positions["ycentroid"]

    dbscan_group = DBSCANGroup(crit_separation=eps)

    dbsc_star_groups = dbscan_group(starlist)

    dbsc_star_groups = dbsc_star_groups.group_by('group_id')

    return dbsc_star_groups 

In [ ]:
# Michael's edge masks 
# threshold value set equal to minimum range of edge mask in standard
# gPhoton pipeline

def mask_margins(header, shape, threshold=1420):
    y, x = np.indices(shape)
    distances = np.sqrt(
        (y - header['CRPIX2']) ** 2 + (x - header['CRPIX1']) ** 2
    )
    return distances > threshold


def make_galex_mask(image_dict):
    mask_ix = np.nonzero(image_dict['flag'] + image_dict['edge'])
    mask = np.full(image_dict['flag'].shape, False)
    mask[mask_ix] = True
    margin = mask_margins(image_dict['header'], image_dict['cnt'].shape)
    mask += margin
    return mask

In [ ]:
# corralling all the data and retroactively calculating total exposure time using the exposure time file (not the
# most efficient way, could get it in the gphoton2 pipeline but...)

folder = "/home/bekah/glcat/exploration_notebooks/test_data"

expTime_dict = {}

from os.path import exists

for eclipse_num in extended_sources_eclipses:
    print(eclipse_num[1])
    if exists(f"/home/bekah/glcat/exploration_notebooks/test_data/e{eclipse_num[1]}/e{eclipse_num[1]}-nd-30s-exptime.csv"):
        exp_time = pd.read_csv(f"/home/bekah/glcat/exploration_notebooks/test_data/e{eclipse_num[1]}/e{eclipse_num[1]}-nd-30s-exptime.csv")
        tot_exp_time = sum(exp_time["expt"])

        expTime_dict[eclipse_num[1]] = tot_exp_time

In [ ]:
expTime_sources = []

for e in expTime_dict: 
    image_data = image_reader(e)
    sources = DAOFindCall(image_data/expTime_dict[e]) 
    #sources = DAOFindCall(image_data) 
    num_sources = len(sources)
    expTime_sources.append((expTime_dict[e],num_sources,e))

In [ ]:
expTime_dict

In [ ]:
expTime_sources

In [ ]:

x_val = [x[0] for x in expTime_sources]
y_val = [x[1] for x in expTime_sources]
ecl_str = [x[2] for x in expTime_sources]

plt.scatter(x_val,y_val)
plt.xlabel("Total Exposure Time")
plt.ylabel("# DAO Sources ID'd")
for i, label in enumerate(ecl_str):
    plt.annotate(label, (x_val[i], y_val[i]))
plt.show()

In [ ]:
eclipse = 581

In [ ]:
image_data = image_reader(eclipse)
dao_sources = DAOFindCall(image_data/654.158)
#dao_sources

In [ ]:
dao_sources = dao_sources[dao_sources['sharpness']>.4]

In [ ]:
fig, axs = plt.subplots(2, 2)

axs[0,0].imshow(centile_clip(image_reader(eclipse),centiles=(0,95)),interpolation='none')
axs[0,0].scatter(dao_sources["xcentroid"], dao_sources["ycentroid"], color="white", alpha=1, s=.5)
axs[0,0].set_title(f"Eclipse {eclipse}")

axs[0,1].hist(dao_sources["roundness1"])
axs[0,1].set_title("roundness1")

axs[1,1].hist(dao_sources["roundness2"])
axs[1,1].set_title("roundness2")

axs[1,0].hist(dao_sources["sharpness"])
axs[1,0].set_title("sharpness")

In [ ]:
DBSCAN_grouped = grouping_sources(dao_sources, 20)

In [ ]:
plt.imshow(centile_clip(image_reader(eclipse),centiles=(0,95)),interpolation='none')

cmap = make_random_cmap(ncolors=5000,seed=34)

for i, group in enumerate(DBSCAN_grouped.groups):
    xypos = np.transpose([group['x_0'], group['y_0']])
    ap = CircularAperture(xypos, r=2)
    ap.plot(color=cmap.colors[i],lw=.8)
plt.show()